In [ ]:
import random
import streamlit as st
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32  # Use FP16 on GPU for speed

# List of optimized open-source models
models = {
    "GPT-Neo": "EleutherAI/gpt-neo-125M",  # Lightweight model
    "Mistral": "mistralai/Mistral-7B-Instruct-v0.1",  # Efficient model
    "Falcon": "tiiuae/falcon-7b"  # Optimized model
}

# Predefined list of random topics
topics = [
    "The importance of space exploration",
    "The impact of climate change on wildlife",
    "The history of the internet",
    "How do vaccines work?",
    "The psychology behind dreams",
    "The rise of electric vehicles",
    "Ancient civilizations and their contributions",
    "The benefits of mindfulness and meditation",
    "How cryptocurrencies are changing the economy",
    "The effects of music on the human brain"
]

# Streamlit UI
st.title("Article Generator using Open-Source LLMs")
st.sidebar.header("Settings")

# Select model
model_name = st.sidebar.selectbox("Choose a model", list(models.keys()))
random_topic = st.sidebar.selectbox("Choose a topic", topics)

@st.cache_resource()
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(models[model_name])
    model = AutoModelForCausalLM.from_pretrained(models[model_name], torch_dtype=torch_dtype).to(device)
    
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    return model, tokenizer

# Load model once
model, tokenizer = load_model(model_name)

# Function to generate an article
def generate_article(model, tokenizer, prompt, max_length=150):  # Reduced max_length for faster generation
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_p=0.9,
        top_k=20,  # Reduced top_k for speed
        temperature=0.8,  # Balanced creativity & speed
        do_sample=True,
        repetition_penalty=1.1,
        pad_token_id=model.config.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

if st.button("Generate Article"):
    st.write(f"### Generating an article on: {random_topic} using {model_name}")
    prompt = f"Write an informative article about {random_topic}."
    article = generate_article(model, tokenizer, prompt)
    st.write(article)
